In [13]:
import os
import pickle
import numpy as np
import matplotlib as mpl

from easydict import EasyDict
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
from scipy.io import loadmat
import scipy.io as sio

In [14]:
# parameter
batch_size = 8
opts = EasyDict()
opts.input_size = 2
opts.output_size = 1
opts.hidden_size = 128
opts.num_layers = 2
opts.nums_epoch = 200
opts.dropout = 0.5 
opts.isBidirectional = True;
opts.layer_size = 2
opts.sequence_length = 300
opts.attention_size = 10
opts.adaTrain = 1
opts.nums_ada_epoch = 5
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

name_labels = ["p1","p2" ]
# ,"noise","differentplace"  "tabish" "john"
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    std = np.std(x)
    mean = np.mean(x)
    x = (x - np.mean(x)) / np.std(x)
    return x,mean,std
file_power_name = []
file_HR_name = []
for i in range(1,11):
    name = 'radar_'+ str(i).rjust(2,'0') + '.csv'
    file_power_name.append(name)
print(file_power_name)
for i in range(1,11):
    name = 'HR'+ str(i).rjust(2,'0') + '.npy'
    file_HR_name.append(name)   
print(file_HR_name)

para_normal_sep = 6
para_normal_jian = 15

['radar_01.csv', 'radar_02.csv', 'radar_03.csv', 'radar_04.csv', 'radar_05.csv', 'radar_06.csv', 'radar_07.csv', 'radar_08.csv', 'radar_09.csv', 'radar_10.csv']
['HR01.npy', 'HR02.npy', 'HR03.npy', 'HR04.npy', 'HR05.npy', 'HR06.npy', 'HR07.npy', 'HR08.npy', 'HR09.npy', 'HR10.npy']


In [187]:
#-*- coding : utf-8-*-

import os
import pandas as pd
import numpy as np
from scipy import interpolate
import random

name_labels = ["p1","p2" ]
file_HR_name = []
for i in range(1,11):
    name = 'HR'+ str(i).rjust(2,'0') + '.csv'
    file_HR_name.append(name)

def readDir(dirPath,pathArray,res,prevPath = ""):
    if dirPath[-1] == '/':
        return
    flag = False
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        fileList.sort()
        for f in fileList:
            if f in file_HR_name:
                flag = True
            file_name = f
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readDir(f,pathArray,res,dirPath)
                allFiles = subFiles + allFiles 
            else:
                if flag:
#                     print(f)
#                     print(prevPath)
                    pathArray.append(f)
                    csv_data = pd.read_csv(f,engine='python')
                    header = np.array(csv_data.columns)
                    
                    if(header[1] == u'数据组 1:心率(bpm)' and header[2] == u'数据组 1:力(N)'):
                        data = np.array(csv_data.iloc[0:301])
                        rr = data[:,3]
                        res_rr = np.delete(rr,np.where(np.isnan(rr)))
                        if((res_rr>10).all() == False or (res_rr<30).all() == False):
                            idx = np.where((res_rr > 10) & (res_rr <30))
                            idx_abnormal = np.where((res_rr < 10) | (res_rr > 30))
                            
                            if len(idx[0]) == 0:
                                temp = 15 + np.random.randn(4)
                                res_rr = np.around(temp, decimals=1)
                            elif len(idx[0]) == 1:
                                temp = res_rr[idx[0]] + np.random.randn(4)
                                res_rr = np.around(temp, decimals=1)
                            else:
                                z = np.polyfit(idx[0],res_rr[idx],1)
                                p= np.poly1d(z)
                                xnew = np.arange(4)
                                ynew = p(xnew)
                                ynew_idx = np.where((ynew < 10) | (ynew > 30))
                                ynew[ynew_idx] = 15 + np.random.randn(len(ynew_idx[0]))
                                res_rr = np.around(ynew, decimals=1)
#                             print('Warnning: the rr is less than 5\n path:'+str(f))
                        res.append(data)
                        res_hr = data[:,1]
                        res_hr = np.delete(res_hr,np.where(np.isnan(res_hr)))
                        xnew = np.linspace(0,3,num= 301)
                        fun_hr=interpolate.interp1d(np.linspace(0,3,num= 31),res_hr,kind="slinear")
                        res_hr = fun_hr(xnew)
                        
                        fun=interpolate.interp1d(np.arange(4),res_rr,kind="slinear")
                        
                        res_rr=fun(xnew)
                        res_rr = np.around(res_rr, decimals=1)
                        path_save_gt = prevPath +"/vitalSign"
                        if not os.path.exists(path_save_gt):
                            os.makedirs(path_save_gt)
                        res_gt = np.vstack((res_hr,res_rr))
                        res_gt = res_gt[:,:-1]
                        
                        path_save_gt = path_save_gt+"/"+os.path.splitext(file_name)[0]+".npy"
                        np.save(path_save_gt,res_gt.T)
                        
        
                    else:
                        print('Failed becaues of index error\n path:'+str(f))
                    allFiles.append(f)
                    flag = False
        return allFiles

for name in name_labels:    
    path_save_dir = "/home/kaixin/RobustVSDataset/aggregated/"+name+"/"
    if not os.path.exists(path_save_dir):
        os.makedirs(path_save_dir) 
    path = "/data2/jian/RobustVSDataset/"+name
    
#     path_save = path_save_dir +'dir.txt'
#     path_save_hr = path_save_dir + 'hr.npy'
#     path_save_power = path_save_dir + 'rr_power.npy'
#     path_save_rr = path_save_dir + 'rr.npy'
    
    pathArray =  []
    resArray = []
    res = readDir(path,pathArray,resArray)
    resArray = np.array(resArray)

#     np.savetxt(path_save,np.array(pathArray),fmt='%s')
#     res_hr = resArray[:,:,1]
#     res_rr = resArray[:,:,3]
#     # res_hr = resArray[(~np.isnan())]
#     #  = 
#     # resArray[:,:,3]
#     res_hr = np.delete(res_hr,np.where(np.isnan(res_hr))[1],axis=1)
#     res_power = resArray[:,:,2];
#     res_rr = np.delete(res_rr,np.where(np.isnan(res_rr))[1],axis=1)
# #     print(res_hr.shape)
# #     print(res_power.shape)
# #     print(res_rr.shape)
#     np.save(path_save_hr,res_hr)
#     np.save(path_save_rr,res_rr)
#     np.save(path_save_power,res_power)

In [15]:
# LSTM model
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size = opts.input_size,hidden_size = opts.hidden_size, num_layers = opts.num_layers,
                                  batch_first = True,bidirectional = opts.isBidirectional,dropout = opts.dropout)
        for name, param in self.lstm.named_parameters():
#             if 'bias' in name:
#                 nn.init.constant_(param, 0.0)
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
        self.out = torch.nn.Linear(opts.hidden_size*2, opts.output_size)
    
    def forward(self, sequence):
        r_out,_ = self.lstm(sequence)
        out = self.out(r_out)
        return out
# read power file
def readPowerDir(dirPath,pathArray,res):
    if dirPath[-1] == '/':
        return
    flag = False
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        fileList.sort()
        for f in fileList:
            if f in file_power_name:
                flag = True
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readPowerDir(f,pathArray,res)
                allFiles = subFiles + allFiles 
            else:
                if flag:
                    
                    pathArray.append(f)
                    csv_data = pd.read_csv(f,engine='python',header=None)
                    data = np.array(csv_data)
                    for i in range(data.shape[0]):
                        data[:,1],_,_ = ZscoreNormalization(data[:,1])
                        data[:,2],_,_ = ZscoreNormalization(data[:,2])
                    res.append(data)
                    allFiles.append(f)
                    flag = False
    return allFiles

# read HR file
def readHRDir(dirPath,pathArray,res):
    if dirPath[-1] == '/':
        return
    flag = False
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        fileList.sort()
        for f in fileList:
            if f in file_HR_name:
                flag = True
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readHRDir(f,pathArray,res)
                allFiles = subFiles + allFiles 
            else:
                if flag:
                    
                    pathArray.append(f)
                    data = np.load(f)
#                     for i in range(data.shape[0]):
#                         data[:,1],_,_ = ZscoreNormalization(data[:,1])
#                         data[:,0],_,_ = ZscoreNormalization(data[:,0])
                    res.append(data)
                    allFiles.append(f)
                    flag = False
    return allFiles



In [16]:
average = []
average_fix = []
average_amb = []
# Cross-validation len(name_labels)
for i in range(len(name_labels)):
    name_mask = []
    name_mask.append(name_labels[i])
    # read power and distance from file according ro name_labels as trainset
    PowerTrain = np.array([0])
    HRTrain = np.array([0])
    for name in name_labels:
        if(name in name_mask):
            continue
        pathArray =  []
        power = []
        HR = []
        path = "/data2/jian/RobustVSDataset/"+name
        res = readPowerDir(path,pathArray,power)
        pathArray =  []
        res = readHRDir(path,pathArray,HR)
        power = np.array(power)
        HR = np.array(HR)
        PowerTrain = power if  PowerTrain.any() == 0 else np.concatenate((PowerTrain, power), axis = 0)
        HRTrain = HR if  HRTrain.any() == 0 else np.concatenate((HRTrain, HR), axis = 0)
    # read power and distance from file according ro name_mask as testset
    PowerTest = np.array([0])
    HRTest = np.array([0])    
    for name in name_mask:
        pathArray =  []
        power = []
        HR = []
        path = "/data2/jian/RobustVSDataset/"+ name
        res = readPowerDir(path,pathArray,power)
        pathArray =  []
        res = readHRDir(path,pathArray,HR)
        power = np.array(power)
        HR = np.array(HR)
        PowerTest = power if  PowerTest.any() == 0 else np.concatenate((PowerTest, power), axis = 0)
        HRTest = HR if  HRTest.any() == 0 else np.concatenate((HRTest, HR), axis = 0)
#     print("Read file success!")
    
    
    model = LSTM()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)   # optimize parameters
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.8,weight_decay=1e-4)
    loss_func = nn.MSELoss()

    inputs, targets = torch.from_numpy(PowerTrain[:,:,1:]).type(torch.FloatTensor),torch.from_numpy(HRTrain[:,:,0]).type(torch.FloatTensor)
    inputs, targets = inputs.to(device), targets.to(device)
    torch_dataset = Data.TensorDataset(inputs,targets)
    loader = Data.DataLoader(
                    dataset=torch_dataset,      # torch TensorDataset format
                    batch_size=batch_size,      # mini batch size
                    shuffle=True,                            
                )
    train_loss, train_count, val_loss, val_count = 0,0,0,0
    val_inputs,val_targets  = torch.from_numpy(PowerTest[:,:,1:]).type(torch.FloatTensor),torch.from_numpy(HRTest[:,:,0]).type(torch.FloatTensor)
    val_inputs,val_targets = val_inputs.to(device), val_targets.to(device)
    x_ada = torch.zeros(val_inputs.shape)
    y_ada = torch.zeros(val_targets.shape)
    print(x_ada.shape,y_ada.shape,val_inputs.shape)
    for ii in range(x_ada.shape[0]):
        for jj in range(50):
            if(opts.adaTrain > 0):
                x_ada[ii,200:,:] = val_inputs[ii,200:,:]
                y_ada[ii,250+jj] = val_targets[ii,-1] 
                y_ada[ii,200+jj] = val_targets[ii,-50] 
#                 y_ada[ii,150+jj] = val_targets[ii,-100]
#                 y_ada[ii,100+jj] = val_targets[ii,-150]
    val_inputs = val_inputs.permute(1, 0, 2)
    val_targets = val_targets.t()
    val_targets = val_targets[:,:,np.newaxis]
    



#   begin train
    for epoch in range(0, opts.nums_epoch+1):
        for step, (x, y) in enumerate(loader):
            
            model.zero_grad()
            model.train()
            x = x.permute(1, 0, 2)
            y = y.t()
            y = y[:,:,np.newaxis]
            prediction= model(x)
            loss = loss_func(prediction, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
            train_loss += loss.item()
            train_count += 1

#         model.eval()
#         with torch.no_grad():
#             val_predict = model(val_inputs)
#             loss_ = loss_func(val_predict, val_targets)
#             val_loss += loss_.item()
#             val_count += 1
#         if(epoch% 50 == 0):
#             print('Epoch %d / %d     Train loss: %f     val loss: %f'% (epoch,opts.nums_epoch,  train_loss/train_count,val_loss/val_count)) 
    
    x_ada,y_ada= x_ada.to(device), y_ada.to(device)
#     print(x_ada)
#     print(y_ada)
    torch_dataset_ada = Data.TensorDataset(x_ada,y_ada)
    
    loader1 = Data.DataLoader(
                    dataset=torch_dataset_ada,      # torch TensorDataset format
                    batch_size=batch_size,      # mini batch size
                    shuffle=True,                            
                )
    for epoch in range(0,opts.nums_ada_epoch+1):
        for step, (x, y) in enumerate(loader1):
            model.zero_grad()
            model.train()
            x = x.permute(1, 0, 2)
            y = y.t()
            y = y[:,:,np.newaxis]
            
            prediction= model(x)
            
            
            loss = loss_func(prediction[200:,], y[200:,])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
    
    model.zero_grad()
    
    val_predict = model(val_inputs)
    sum = 0
    sum_fix = 0
    sum_amb = 0
    for i in range(val_predict.shape[1]):
        if name_mask[0] == "jian":
            para = para_normal_jian
        else:
            para = para_normal_sep
        if(i < para):
            sum_amb += torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300
        else:
            sum_fix += torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300
        sum += torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300
        print('index:%d    MSE: %f '%(i,torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300))
    print('Test: %s  average MSE: %f average fix MSE: %f  average amb MSE: %f' % (name_mask[0],sum/val_predict.shape[1],
                                                                              sum_fix/(val_predict.shape[1]-para),sum_amb/para))
    average.append((sum/val_predict.shape[1]).item())
    average_fix.append((sum_fix/(val_predict.shape[1]-para)).item())
    
    average_amb.append((sum_amb/(para)).item())
print("average: %f" %(np.mean(average)))
print("average fix: %f" %(np.mean(average_fix)))
print("average amb: %f" %(np.mean(average_amb)))
print(average)

np.savetxt("generation_ada_2point.csv", average)
print(average_fix)
print(average_amb)

torch.Size([18, 300, 2]) torch.Size([18, 300]) torch.Size([18, 300, 2])
index:0    MSE: 2.803355 
index:1    MSE: 2.270577 
index:2    MSE: 2.301537 
index:3    MSE: 1.565176 
index:4    MSE: 3.598841 
index:5    MSE: 4.066598 
index:6    MSE: 6.582900 
index:7    MSE: 2.504412 
index:8    MSE: 6.811568 
index:9    MSE: 4.228043 
index:10    MSE: 1.364400 
index:11    MSE: 3.417266 
index:12    MSE: 3.673333 
index:13    MSE: 3.159280 
index:14    MSE: 3.194962 
index:15    MSE: 4.374754 
index:16    MSE: 8.613933 
index:17    MSE: 8.084437 
Test: kaixin  average MSE: 4.034187 average fix MSE: 4.667441  average amb MSE: 2.767681
torch.Size([18, 300, 2]) torch.Size([18, 300]) torch.Size([18, 300, 2])
index:0    MSE: 9.238263 
index:1    MSE: 8.659771 
index:2    MSE: 7.249311 
index:3    MSE: 8.281296 
index:4    MSE: 13.171264 
index:5    MSE: 6.867240 
index:6    MSE: 8.904328 
index:7    MSE: 7.685376 
index:8    MSE: 14.933533 
index:9    MSE: 9.649515 
index:10    MSE: 6.437246 
in

index:0    MSE: 2.083045 
index:1    MSE: 0.850974 
index:2    MSE: 2.506006 
index:3    MSE: 4.720536 
index:4    MSE: 2.377388 
index:5    MSE: 1.390704 
index:6    MSE: 5.001315 
index:7    MSE: 5.227843 
index:8    MSE: 3.624907 
index:9    MSE: 3.388038 
index:10    MSE: 4.121620 
index:11    MSE: 3.382268 
index:12    MSE: 4.878640 
index:13    MSE: 3.862948 
index:14    MSE: 4.217146 
index:15    MSE: 4.278006 
index:16    MSE: 2.603887 
index:17    MSE: 6.338726 
Test: chen  average MSE: 3.603000 average fix MSE: 4.243779  average amb MSE: 2.321442
torch.Size([18, 300, 2]) torch.Size([18, 300]) torch.Size([18, 300, 2])
index:0    MSE: 9.921135 
index:1    MSE: 10.272963 
index:2    MSE: 8.231771 
index:3    MSE: 12.503972 
index:4    MSE: 10.258164 
index:5    MSE: 13.713510 
index:6    MSE: 12.947553 
index:7    MSE: 4.383501 
index:8    MSE: 17.767998 
index:9    MSE: 18.261978 
index:10    MSE: 6.165503 
index:11    MSE: 2.912947 
index:12    MSE: 12.669505 
index:13    MSE: